In [1]:
# import sys
# sys.path.append('../tools')

In [2]:
from fastai.text.all import *
from transformers import (BertTokenizer, AutoModel)
# from train_utils import *
from tools.train_utils import *
from madgrad import MADGRAD

In [3]:
base_path = 'data'
# base_path = '../data'

BERT_PATH = base_path + '/bert-indo-15g'

bert_model_file = base_path + '/saved_models/text_model/fine_tune_5epoch_bertindo15g-bs-64-mgrad-m06-08.pth'

image_model_file = base_path + '/saved_models/fine_tune_5epoch_nfnetl0-bs-64-pre-mgrad-adaptconcat-embed-1024-m08-10.pth'

In [5]:
OUTPUT_CLASSES = 11014
train_df = add_splits(pd.read_csv(base_path +'/train.csv'))


/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [5]:
# class ArcFaceLoss(Module):
#     y_int=True
#     def __init__(self, m: float = 0.5, s: int =30, output_classes: int =11014, weight=None, reduction='mean'):
#         self.m=m 
#         self.s=s 
#         self.output_classes=output_classes
#         self.weight=weight 
#         self.reduction=reduction
    
#     def forward(self, cosine, targ):
#         cosine = cosine.clip(-1+1e-7, 1-1e-7) 
#         arcosine = cosine.arccos()
#         arcosine += F.one_hot(targ, num_classes = self.output_classes) * self.m
#         cosine2 = arcosine.cos()
#         cosine2 *= self.s
#         return F.cross_entropy(cosine2, targ)

#     def set_margin(self, new_m):
#         self.m = new_m

# class ArcFaceLossFlat(BaseLoss):
#     y_int = True 
#     def __init__(self, *args, m: float = 0.5, s: int =30, output_classes: int =11014, weight=None, reduction='mean', axis=-1):
#         super().__init__(ArcFaceLoss, *args, m=m, s=s, output_classes=output_classes, weight=weight, reduction=reduction)
        
#     def decodes(self, x): 
#         return x.argmax(dim=self.axis)
    
#     def activation(self, x): 
#         return F.softmax(x, dim=self.axis)
    
# class ArcFaceClassifier(nn.Module):
#     def __init__(self, in_features, output_classes):
#         super().__init__()
#         self.initial_layers=nn.Sequential(
#             nn.BatchNorm1d(in_features),
#             nn.Dropout(.25))
#         self.W = nn.Parameter(torch.Tensor(in_features, output_classes))
#         nn.init.kaiming_uniform_(self.W)
#     def forward(self, x):
#         x = self.initial_layers(x)
#         x_norm = F.normalize(x)
#         W_norm = F.normalize(self.W, dim=0)
#         return x_norm @ W_norm
    
class BertArcFace(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert_model = AutoModel.from_pretrained(BERT_PATH)
        self.classifier = ArcFaceClassifier(768, OUTPUT_CLASSES)
        self.outputEmbs = False
    def forward(self, x):
        output = self.bert_model(*x)
        embeddings =output.last_hidden_state[:,0,:]
        if self.outputEmbs:
            return [self.classifier(embeddings), embeddings]
        return self.classifier(embeddings)
    
# class MarginScheduler(Callback):
#     def __init__(self, start_m, end_m):
#         self.start_m = start_m
#         self.end_m = end_m

#     def after_create(self):
#         try:
#             if self.learn.loss_func is None : 
#                 raise ValueError
#             if getattr(self.learn.loss_func, 'm', False) is False :
#                 raise ValueError
#         except ValueError as e:
#             print( e," Loss function not defined or loss function has no margin attribute.")

#     def before_fit(self):
#         if self.learn.n_epoch > 1:
#             step = (self.end_m - self.start_m) / (self.learn.n_epoch -1)
#             self.margins = np.arange(self.start_m, self.end_m + step,step)
#         else:
#             self.learn.loss_func.set_margin(self.end_m)

#     def before_epoch(self):
#         if self.learn.epoch > 0:
#             self.learn.loss_func.set_margin(self.margins[self.learn.epoch])
            
# class F1FromEmbss(Callback):
#     def after_pred(self):
#         if not self.training:
#             self.embs.append(self.learn.pred[1])
#             self.ys.append(self.learn.yb[0])
#             self.learn.pred = self.learn.pred[0]
#     def before_validate(self):
#         self.ys = []
#         self.embs = []
#         self.model.outputEmbs = True
#     def before_train(self):
#         self.model.outputEmbs = False
#     def after_validate(self):
#         embs = torch.cat(self.embs)
#         embs = F.normalize(embs)
#         ys = torch.cat(self.ys)
#         score = f1_from_embs(embs,ys)
#         self.learn.metrics[0].val = score

In [6]:
#Taken from https://www.kaggle.com/c/shopee-product-matching/discussion/233605#1278984
# def string_escape(s, encoding='utf-8'):
#     return s.encode('latin1').decode('unicode-escape').encode('latin1').decode(encoding)

# class TitleTransform(Transform):
#     def __init__(self):
#         super().__init__()
#         self.tokenizer = BertTokenizer.from_pretrained(BERT_PATH)
               
#     def encodes(self, row):
#         text = row.title
#         text=string_escape(text)
#         encodings = self.tokenizer(text, padding = 'max_length', max_length=100, truncation=True,return_tensors='pt')
#         keys =['input_ids', 'attention_mask']
#         return tuple(encodings[key].squeeze() for key in keys)

# def get_text_dls(bs=64):
#     tfm = TitleTransform()

#     data_block = DataBlock(
#         blocks = (TransformBlock(type_tfms=tfm), 
#                   CategoryBlock(vocab=train_df.label_group.to_list())),
#         splitter=ColSplitter(),
#         get_y=ColReader('label_group'),
#         )
#     return  data_block.dataloaders(train_df, bs=bs)

def split_2way(model):
    return L(params(model.bert_model), params(model.classifier))

In [7]:
class CONF( ConfigClass):
    arcface_m = .5
    arcface_s = 30.0
    lr = 1e-2
    lr_mult = 100.0
    n_epoch = 5
    start_m = 0.6
    end_m = 0.8
    bs = 64
    gradient_clip=1
    experiment_id=1
    val_k=0
conf = CONF()

In [8]:
loss_func = ArcFaceLoss()
opt_func = partial(OptimWrapper, opt=MADGRAD)
f1_tracker = TrackerCallback(monitor='F1 embeddings', comp=np.greater)

In [9]:
learn = Learner(get_text_dls(train_df, BertTokenizer.from_pretrained(BERT_PATH), conf.bs), BertArcFace(), splitter=split_2way, opt_func=opt_func,loss_func=loss_func, 
                cbs = [ GradientClip,MixedPrecision, MarginScheduler(conf.start_m, conf.end_m),  F1FromEmbs, f1_tracker], 
                metrics=[F1EmbedMetric(), accuracy])
 

Some weights of the model checkpoint at ../input/bertindo15g were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:

learn.fine_tune(conf.n_epoch,conf.lr, freeze_epochs = 1)

epoch,train_loss,valid_loss,F1 embeddings,accuracy,time
0,26.854330,34.571476,0.773117,0.000000,03:18


epoch,train_loss,valid_loss,F1 embeddings,accuracy,time
0,22.452049,35.156628,0.795860,0.000000,04:34
1,16.396692,32.769279,0.813829,0.000000,04:33
2,15.079148,35.089069,0.817722,0.000000,04:34
3,14.134083,36.763489,0.820931,0.000000,04:34
4,14.083672,38.009205,0.821997,0.000000,04:34


In [11]:
saved_name  = f'fine_tune_{conf.n_epoch}epoch_bertindo15g-bs-{conf.bs}-mgrad-m{(int(conf.start_m *10)):02d}-{int(conf.end_m *10):02d}'

In [12]:
learn.save('stage1-'+ saved_name)

Path('models/stage1-fine_tune_5epoch_bertindo15g-bs-64-mgrad-m06-08.pth')

In [13]:
def save_without_classifier(model, fname):
    model.classifier = None
    torch.save(model.state_dict(), fname)

In [14]:
print('saving',f'model' )
save_without_classifier(learn.model,saved_name)

saving model
